# check oof df

In [1]:
import os
import gc
import yaml
import argparse

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch

from tqdm.notebook import tqdm

import warnings
warnings.filterwarnings('ignore')

from sklearn.metrics.pairwise import cosine_similarity 
import sys
sys.path.append("/kaggle/src/dss_utils")
sys.path.append("/kaggle/src/exp")
sys.path.append("/kaggle/src/model")
sys.path.append("/kaggle/src/data")
from dss_metrics import score
from training_loop import get_valid_values_dict, concat_valid_input_info, get_oof_df, get_key_df
from dss_model import get_model
from dss_dataloader import get_loader


In [2]:
output_dir = "/kaggle/working"
# exp_name = "exp022_dense_skffolds_epoch10"
# exp_name = "exp023_alldata_skf_dense_epoch30"
# exp_name = "exp025_alldata_skf_lstmenchead_epoch30"
# exp_name = "exp028_lstm_enchead_enckernelto48_epoch30"
# exp_name = "exp028_lstm_enchead_enckernelto96_epoch30"
# exp_name = "exp028_lstm_enchead_enckernelto192_epoch30"
exp_name = "exp037_elapseddate_epoch30"
folds = [0, 1, 2, 3, 4]
# folds = [0]
oof_df = pd.DataFrame()
for i in folds:
    print("fold", i)
    # df = pd.read_parquet(os.path.join(output_dir, "_oof", exp_name, f"raw_oof_df_fold{i}.parquet"))
    df = pd.read_parquet(os.path.join(output_dir, "_oof", exp_name, f"fold{i}_best_oof_df.parquet"))
    oof_df = pd.concat([oof_df, df], axis=0)


print(len(oof_df["series_id"].unique()))   


fold 0
fold 1
fold 2
fold 3
fold 4
277


In [3]:
oof_df = oof_df.sort_values(["series_id", "step"])
display(oof_df)

oof_df = oof_df[["series_id", "series_date_key","step", "second", "event_pred", "class_pred", "anglez"]]


,series_id,step,anglez,enmo,elapsed_date,second,date,anglez_mean_36,anglez_std_36,anglez_mean_60,anglez_std_60,event,event_onset,event_wakeup,series_date_key,series_date_key_str,fold,event_pred,class_pred,class_target
0,038441c925bb,0.0,2.636700,0.0217,0.0,0,2018-08-14,0.0,0.0,0.0,0.0,0.0,0,0,0,038441c925bb_2018-08-14,2,0.222598,0.251788,0.0
1,038441c925bb,1.0,2.636800,0.0215,0.0,5,2018-08-14,0.0,0.0,0.0,0.0,0.0,0,0,0,038441c925bb_2018-08-14,2,0.000000,0.195168,0.0
2,038441c925bb,2.0,2.637000,0.0216,0.0,10,2018-08-14,0.0,0.0,0.0,0.0,0.0,0,0,0,038441c925bb_2018-08-14,2,0.000000,0.152249,0.0
3,038441c925bb,3.0,2.636800,0.0213,0.0,15,2018-08-14,0.0,0.0,0.0,0.0,0.0,0,0,0,038441c925bb_2018-08-14,2,0.000000,0.128238,0.0
4,038441c925bb,4.0,2.636800,0.0215,0.0,20,2018-08-14,0.0,0.0,0.0,0.0,0.0,0,0,0,038441c925bb_2018-08-14,2,0.000000,0.115376,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25500432,fe90110788d2,592375.0,-27.277500,0.0204,1.0,35,2017-09-08,0.0,0.0,0.0,0.0,-1.0,0,0,7745,fe90110788d2_2017-09-08,3,0.000000,0.000876,-1.0
25500433,fe90110788d2,592376.0,-27.032499,0.0233,1.0,40,2017-09-08,0.0,0.0,0.0,0.0,-1.0,0,0,7745,fe90110788d2_2017-09-08,3,0.000000,0.000877,-1.0
25500434,fe90110788d2,592377.0,-26.841200,0.0202,1.0,45,2017-09-08,0.0,0.0,0.0,0.0,-1.0,0,0,7745,fe90110788d2_2017-09-08,3,0.000000,0.000876,-1.0
25500435,fe90110788d2,592378.0,-26.723900,0.0199,1.0,50,2017-09-08,0.0,0.0,0.0,0.0,-1.0,0,0,7745,fe90110788d2_2017-09-08,3,0.000000,0.000876,-1.0


# nan detect

In [4]:
def detect_similarity_data(anglez, before_date_anglez, cut_window_num=4, only_cos_sim_thr=0.995):
    window_size = int(anglez.shape[1]/cut_window_num)
    count = 0
    for window_idx in range(cut_window_num):
        anglez_window = anglez[:,window_idx*window_size:(window_idx+1)*window_size]
        before_date_anglez_window = before_date_anglez[:, window_idx*window_size:(window_idx+1)*window_size]
        anglez_simirality = cosine_similarity(anglez_window, before_date_anglez_window)
        # only cos sim以上のsimilarityになるか、std_cos_sim以上のsimilarityかつstd_thr以下
        if anglez_simirality > only_cos_sim_thr:
            count += 1
    # 検出モデルに入力すべきものを1、スルーして無検出とすべきものを0にする
    detect_target = 1 - int(count > 0)
    return detect_target


In [5]:
def get_remove_target_keys(series_df, series_keys):
    remove_target_keys = []
    before_date_anglez = np.zeros((1, 1))
    loader = tqdm(enumerate(series_keys), total=len(series_keys))
    for idx, key in loader:
        # target_series = series_df.filter(pl.col("key")==key)
        target_series = series_df[series_df["series_date_key"]==key]
        # anglez = np.array(target_series.get_column("anglez")).reshape(1,-1)
        anglez = np.array(target_series["anglez"]).reshape(1,-1)
        if anglez.shape[1]==before_date_anglez.shape[1]:
            detect_pred = detect_similarity_data(anglez, before_date_anglez)
        else:
            detect_pred = 1
        if detect_pred==0:
            remove_target_keys.append(key)
        before_date_anglez = anglez
    return remove_target_keys


In [6]:
oof_df


,series_id,series_date_key,step,second,event_pred,class_pred,anglez
0,038441c925bb,0,0.0,0,0.222598,0.251788,2.636700
1,038441c925bb,0,1.0,5,0.000000,0.195168,2.636800
2,038441c925bb,0,2.0,10,0.000000,0.152249,2.637000
3,038441c925bb,0,3.0,15,0.000000,0.128238,2.636800
4,038441c925bb,0,4.0,20,0.000000,0.115376,2.636800
...,...,...,...,...,...,...,...
25500432,fe90110788d2,7745,592375.0,35,0.000000,0.000876,-27.277500
25500433,fe90110788d2,7745,592376.0,40,0.000000,0.000877,-27.032499
25500434,fe90110788d2,7745,592377.0,45,0.000000,0.000876,-26.841200
25500435,fe90110788d2,7745,592378.0,50,0.000000,0.000876,-26.723900


In [7]:
# series_keys = oof_df["series_date_key"].unique()
# # series_df = pl.DataFrame(oof_df)

# remove_target_keys = get_remove_target_keys(oof_df[["series_date_key", "anglez"]], series_keys)


In [8]:
# remove_target_keys


In [9]:
# pd.DataFrame({"remove_target_keys":remove_target_keys}).to_csv("/kaggle/working/remove_target_keys.csv", index=False)


In [13]:
remove_target_keys = pd.read_csv("/kaggle/working/remove_target_keys.csv")["remove_target_keys"].values.tolist()


In [15]:
len(remove_target_keys)


13

# predict

In [16]:
# postprocess_fn
# series_idでgroupbyして、class_predに対して対象の列のデータから前のN個の列までのデータの平均をとる
import torch
import torch.nn as nn

def detect_event_from_classpred(df, N=300, maxpool_kernel_size=41, remove_target_keys=None):
    df = df.copy()

    # series_idでgroupbyして、class_predに対して対象の列のデータから
    # 前のN個の列までのデータの平均をとる
    df["class_pred_beforemean"] = df.groupby("series_id")["class_pred"].apply(
        lambda x: x.rolling(N, min_periods=1).mean()
    )
    df["class_pred_aftermean"] = df.groupby("series_id")["class_pred"].apply(
        lambda x: x[::-1].rolling(N, min_periods=1).mean()[::-1]
    )
    df["event_pred"] = df["class_pred_beforemean"] - df["class_pred_aftermean"]

    # 入力サイズと出力サイズが一致するようにpaddingを調整
    maxpool_padding = int((maxpool_kernel_size - 1) / 2)
    # maxpoolしてピーク検出
    max_pooling = nn.MaxPool1d(
        maxpool_kernel_size, stride=1, padding=maxpool_padding
    )
    event_pred = df["event_pred"].values
    event_pred = torch.tensor(event_pred).unsqueeze(0)
    pooled_event_pred = max_pooling(np.abs(event_pred)).squeeze(0).numpy()
    event_pred = event_pred.squeeze(0).numpy()
    # peakのところだけ残すmaskを作成
    peak_event_pred_mask = np.where(pooled_event_pred == np.abs(event_pred), 1, 0)
    peak_event_pred = event_pred * peak_event_pred_mask
    df["event_pred"] = peak_event_pred
    df.loc[df["series_date_key"].isin(remove_target_keys), "event_pred"] = 0
    df.loc[df["step"] < 12, "event_pred"] =0
    df["onset"] = np.clip(-df["event_pred"], 0, 1)
    df["wakeup"] = np.clip(df["event_pred"], 0, 1)
    df = df.drop(["class_pred_beforemean", "class_pred_aftermean"], axis=1)

    return df

def make_submission_df(df, threshold=0.1):
    df = df[["series_id", "step", "event_pred"]].copy()
    # thresholdより大きいときは1,-thresholdより小さいときは-1,それ以外は0
    df["event"] = df["event_pred"].apply(lambda x: 1 if x > threshold else -1 if x < -threshold else 0)
    df = df[df["event"] != 0].copy()
    df["event"] = df["event"].replace({1: "wakeup", -1: "onset"})
    df["score"] = df["event_pred"].apply(lambda x: np.clip(np.abs(x), 0.0, 1.0))
    return df


In [17]:
train_event_df = pd.read_csv("/kaggle/input/child-mind-institute-detect-sleep-states/train_events.csv")
train_event_df = train_event_df[train_event_df["step"].notnull()].copy()
# train_event_df = train_event_df[train_event_df["series_id"].isin(oof_df["series_id"].unique())].copy()
train_event_df = train_event_df.reset_index(drop=True)
display(train_event_df.head())


,series_id,night,event,step,timestamp
0,038441c925bb,1,onset,4992.0,2018-08-14T22:26:00-0400
1,038441c925bb,1,wakeup,10932.0,2018-08-15T06:41:00-0400
2,038441c925bb,2,onset,20244.0,2018-08-15T19:37:00-0400
3,038441c925bb,2,wakeup,27492.0,2018-08-16T05:41:00-0400
4,038441c925bb,3,onset,39996.0,2018-08-16T23:03:00-0400


In [18]:
# %%time
# max_kernel_size = 23
# tmp_score = 0
# for average_size_ in range(96, 301, 12):
#     print("average_size", average_size_, "max_kernel_size", max_kernel_size)
#     df = detect_event_from_classpred(oof_df, N=average_size_, maxpool_kernel_size=max_kernel_size, remove_target_keys=remove_target_keys)
#     sub_df = make_submission_df(df, threshold=0.1)
#     if len(sub_df) == 0:
#         print("event not detected")
#         continue
#     print(len(sub_df))
#     score_ = score(train_event_df, sub_df)
#     print(score_)
#     if tmp_score > score_:
#         break
#     tmp_score = score_
#     average_size = average_size_


In [19]:
# tmp_score = 0
# max_kernel_size_ = max_kernel_size
# for max_kernel_size in range(29, 47, 2):
#     print("average_size", average_size, "max_kernel_size", max_kernel_size)
#     df = detect_event_from_classpred(oof_df, N=average_size, maxpool_kernel_size=max_kernel_size, remove_target_keys=remove_target_keys)
#     sub_df = make_submission_df(df, threshold=0.1)
#     if len(sub_df) == 0:
#         print("event not detected")
#         continue
#     score_ = score(train_event_df, sub_df)
#     print(score_)
#     if tmp_score > score_:
#         break
#     tmp_score = score_
#     max_kernel_size_ = max_kernel_size


In [20]:
average_size = 132
max_kernel_size_ = 33

df = detect_event_from_classpred(oof_df, N=average_size, maxpool_kernel_size=max_kernel_size_, remove_target_keys=remove_target_keys)
sub_df = make_submission_df(df, threshold=0.01)
print(average_size, max_kernel_size_)

if len(sub_df) == 0:
    print("event not detected")
    score_ = 0
else:
    score_ = score(train_event_df, sub_df)
print(score_)


132 33
0.7446507192659417


In [ ]:
df = df[["series_id", "step", "class_pred", "onset", "wakeup"]].copy()
df = df.rename(columns={"class_pred": "sleep"})


In [ ]:
print(df.columns)
os.makedirs(os.path.join(output_dir, "_pooled_oof", exp_name), exist_ok=True)
df.to_parquet(os.path.join(output_dir, "_pooled_oof", exp_name, "remove_nan_oof_df.parquet"))


Index(['series_id', 'step', 'sleep', 'onset', 'wakeup'], dtype='object')


exp023_alldata_skf_dense_epoch30

132 41

0.7210358824960511